<a href="https://colab.research.google.com/github/clara-lan/Capstone/blob/main/CapstoneSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use SVM to detect empty space and pre-process images

# Step1: Import system and dependancies

In [ ]:
import os
from os import listdir
from os.path import isfile, join
!pip install roboflow
from roboflow import Roboflow
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
print("TensorFlow version:", tf.__version__)


In [2]:
import IPython.display as display

In [3]:
import pandas as pd
import cv2
import os
import sys
import math
import time
from sklearn import svm
from sklearn.metrics import accuracy_score
from skimage.feature import hog

## Import dataset from Roboflow

In [4]:
#Read method1: Load roboflow dataset in tfrecord
import tensorflow.python.platform
from tensorflow.python.platform import gfile

rf = Roboflow(api_key="cKIuGvQRsLbBvFgxNztc")
project = rf.workspace("myworkspace-nfnwm").project("capstone-nyjby")
dataset = project.version(2).download("tfrecord")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Capstone-2 in tfrecord:: 100%|██████████| 11/11 [00:00<00:00, 42.02it/s]


In [5]:
def extract_images(path):
  #   # Create a description of the features.
  images = []
  features = {
      "image/encoded": tf.io.FixedLenFeature([], tf.string),
      "image/format": tf.io.FixedLenFeature([], tf.string),
      "image/height": tf.io.FixedLenFeature([], tf.int64),
      "image/object/bbox/xmax": tf.io.VarLenFeature(tf.float32),
      "image/object/bbox/xmin": tf.io.VarLenFeature(tf.float32),
      "image/object/bbox/ymax": tf.io.VarLenFeature(tf.float32),
      "image/object/bbox/ymin": tf.io.VarLenFeature(tf.float32),
      "image/object/class/label": tf.io.VarLenFeature(tf.int64),
      "image/object/class/text": tf.io.VarLenFeature(tf.string),
      "image/width": tf.io.FixedLenFeature([], tf.int64),
  }

  train_dataset = tf.data.TFRecordDataset(path)
  # Check an example data in file
  for raw_record in train_dataset.take(144):
    # print(raw_record)
    sample = tf.io.parse_single_example(raw_record, features)
    image = tf.image.decode_image(sample['image/encoded'], dtype=tf.float32) 
    label = sample['image/object/class/label']
    # from matplotlib import pyplot as plt
    # plt.imshow(image, interpolation='nearest')
    # plt.show()
    images.append(image)
  return images

# Step2: Preprocess data

In [6]:
# test_dataset = tf.data.TFRecordDataset('/content/Empty-shelf-detection-15/test/Empty-space.tfrecord')
# valid_dataset = tf.data.TFRecordDataset('/content/Empty-shelf-detection-15/valid/Empty-space.tfrecord')
train_labels = '/content/Capstone-2/train/empty-shelf-space_label_map.pbtxt'
# valid_labels = '/content/Empty-shelf-detection-15/valid/Empty-space_label_map.pbtxt'

In [29]:
label_map_path = '/content/Capstone-2/valid/empty-shelf-space_label_map.pbtxt'
def read_label_map(label_map_path):

    item_id = None
    item_name = None
    items = {}
    
    with open(label_map_path, "r") as file:
        for line in file:
            line.replace(" ", "")
            if line == "item{":
                pass
            elif line == "}":
                pass
     
            elif "name" in line:
                item_name = line.split(":", 1)[1].replace("\"", "").strip()[:-1]
            elif "id" in line:
              item_id = int(line.split(":", 1)[1].strip()[:-1])

            if item_id is not None and item_name is not None:
                # if(item_name == "Void"):
                #    item_name = "empty_space"
                items[item_name] = item_id
                item_id = None
                item_name = None
    return items
read_label_map(label_map_path)

{'empty_shelf': 1, 'object': 2}

# Step3: Train SVM
---



In [52]:
rnoMapping = {}

def datagen(mode):
    """
    Function: datagen 
    
    Input: 
        mode - 1 denotes train data ; 2 denotes test data
    
    Output: Train/Test data and labels depending on mode value
    
    Description: This function computes HOG features for each image in the Dataset/train or Dataset/test folder, assigns label to the descriptor vector of the image and returns the final train/test data and labels matrices used for feeding the SVM in training phase or predicting the label of test data.
    
    """

    X = []
    y = []
    # TODO: remove the dummy index to select y
    idx = 0
    cnt = 0
    if mode == 1:
        path = '/content/Capstone-2/train/empty-shelf-space.tfrecord'
    elif mode == 2:
        path = '/content/Capstone-2/valid/empty-shelf-space.tfrecord'
    # read image
    images = extract_images(path)
    for img in images:
        idx += 1
        # compute HOG features
        des, hog_image = hog(img, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(4, 4), block_norm= 'L2',visualize=True)
        # print(des)

        """Read labels"""
        labels = list(read_label_map(train_labels).keys())
                
        if mode == 1:
            # construct dictionary for roll no. mapping
            for label in labels:
              if label not in rnoMapping.keys():
                  rnoMapping[label] = cnt
                  cnt += 1

        # append descriptor and label to train/test data, labels
        X.append(des)
        # print(labels)
        if(idx % 2 == 0):
          y.append(rnoMapping[labels[0]])
        else:
          y.append(rnoMapping[labels[1]])

    # return data and label
    return X, y
# datagen(1)
# datagen(2)


In [53]:
def train_svm():
    # list of training and test files
    # files_train = [(os.getcwd() + '/Dataset/train/' + f) for f in os.listdir(os.path.join('./Dataset/','train'))]
    # files_test = [(os.getcwd() + '/Dataset/test/' + f) for f in os.listdir(os.path.join('./Dataset/','test'))]

    # call 'datagen' function to get training and testing data & labels
    Xtrain, ytrain = datagen(1)
    Xtest, ytest = datagen(2)

    # convert all matrices to numpy array for fast computation
    Xtrain = np.array(Xtrain)
    ytrain = np.array(ytrain)
    print("x: ", Xtrain.shape)
    print("ytrain: ", ytrain.shape)
    Xtest = np.array(Xtest)
    ytest = np.array(ytest)

    # training phase: SVM , fit model to training data ------------------------------
    clf = svm.SVC(kernel = 'linear')
    clf.fit(Xtrain, ytrain)
    # predict labels for test data
    ypred = clf.predict(Xtest)
    
    # compute accuracy
    accuracy = accuracy_score(ytest, ypred) * 100
    print("\nAccuracy: %.2f" % accuracy + "%")
start_time = time.time()
train_svm()

x:  (144, 67712)
ytrain:  (144,)

Accuracy: 51.39%


In [54]:

print('Execution time: %.2f' % (time.time() - start_time) + ' seconds\n')

Execution time: 128.89 seconds

